In [ ]:
import numpy as np
import tensorflow as tf
import glob
from keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []

for f in glob.glob("images/*/*/*"):
    img_data = tf.io.read_file(f)
    img_data = tf.io.decode_jpeg(img_data)

    # Ensure the image has 3 channels (RGB)
    if img_data.shape[-1] != 3:
        continue

    img_data = tf.image.resize(img_data, [100, 100])

    if f.split("/")[1] == "train":
        X_train.append(img_data)
        y_train.append(int(f.split("/")[2].split("_")[0]))
    elif f.split("/")[1] == "test":
        X_test.append(img_data)
        y_test.append(int(f.split("/")[2].split("_")[0]))


In [ ]:
X_train = np.array(X_train) / 255.0
y_train = np.array(y_train)
X_test = np.array(X_test) / 255.0
y_test = np.array(y_test)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(100, 100, 3)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation="softmax")
])

In [ ]:
callbacks = [EarlyStopping(monitor="loss", patience=1000, min_delta=0.0001)]

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=1000, callbacks=callbacks)

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, callbacks=callbacks)

In [ ]:
sns.set(context="talk")
fig = plt.subplots(figsize=(12, 6))
plt.plot(history.history['loss'])

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.predict(X_test)

In [ ]:
np.argmax(model.predict(X_test), axis=-1)

In [ ]:
import os

In [ ]:
i = 0

for f in glob.glob("division/*.jpeg"):
    KEEP = []
    img_data = tf.io.read_file(f)
    img_data = tf.io.decode_jpeg(img_data)
    # Ensure the image has 3 channels (RGB)
    if img_data.shape[-1] != 3:
        continue
    img_data = tf.image.resize(img_data, [100, 100])
    KEEP.append(img_data)
    KEEP = np.array(KEEP) / 255

    pred = np.argmax(model.predict(KEEP), axis=-1)

    i += 1

    if pred == 0:
        os.rename(f, "storage/printing_board/printing_board" + str(i) + ".jpeg")
    else:
        os.rename(f, "storage/human/human" + str(i) + ".jpeg")